# GLM


- GLM example as simple multivariate posterior estimation problem with available ground-truth (MCMC)
- setup and code from NIPS 2017 paper

In [ ]:
import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.utils.io as io
import delfi.summarystats as ds
import lfimodels.glm.utils as utils
import matplotlib.pyplot as plt
import numpy as np

from lfimodels.glm.GLM import GLM
from lfimodels.glm.GLMStats import GLMStats
from delfi.utils.viz import plot_pdf

import sys
import timeit

import snl.inference.mcmc as mcmc
from lfimodels.snl_exps.util import calc_all_mmds

import snl.inference.diagnostics.two_sample as two_sample
from snl.util import math

import snl.inference.nde as nde
from snl.ml.models.mafs import ConditionalMaskedAutoregressiveFlow
from delfi.utils.delfi2snl import SNLprior, SNLmodel

%matplotlib inline


# load setup from NIPS 2017 paper

len_filter = 9 # number of GLM filter parameters (= dim. of parameters)
duration = 100 # simulation length (longer = tighter posteriors)

true_params, labels_params = utils.obs_params(len_filter)

# basic approach to controlling generator seeds
def init_g(seed):
    m = GLM(seed=seed, duration=duration, len_filter=len_filter)
    #m.I = I
    #p = dd.Uniform(lower=[-3,-3,-3,-3,-3,-3,-3,-3,-3,-3], upper=[3,3,3,3,3,3,3,3,3,3])
    p = utils.smoothing_prior(n_params=m.n_params, seed=seed)
    s = GLMStats(n_summary=m.n_params)
    return dg.Default(model=m, prior=p, summary=s)

# SNPE parameters

# training schedule
n_train=5000
n_rounds=5

# fitting setup
minibatch=100
epochs=1000

# network setup
n_hiddens=[50,50]
reg_lambda=0.01

# convenience
pilot_samples=1000
svi=False
verbose=True
prior_norm=False
val_frac = 0.1

# MAF parameters
mode='random' # ordering of variables for MADEs
n_mades = 5 # number of MADES
act_fun = 'tanh'
batch_norm = False # batch-normalization currently not supported
train_on_all = True # now supported feature

# MDN parameters
n_components = 8

# number of samples for MMD computation
N = 5000

def calc_all_mmds_snl(samples_true, samples_snl):
    """ only called for 'Gaussian' simulator """

    all_mmds = []
    ct = 0
    for samples in samples_snl:
        
        ct += 1
        
        print('\n round #' + str(ct) + '/' + str(len(samples_snl)))
        
        if np.any(np.isnan(samples)): # fail to sample n_sample times
            all_mmds.append(np.inf)
        else:            
            print('- computing MMD')
            scale = math.median_distance(samples_true)
            mmd = two_sample.sq_maximum_mean_discrepancy(samples, samples_true, scale=scale)
            if isinstance(mmd, np.ndarray):
                mmd = mmd.flatten()[0]
            all_mmds.append(mmd)
            
    return np.array(all_mmds).flatten()


In [ ]:

for seed in range(42, 52):
    
    
    observation_seed = seed # draw own xo for each seed (same ground-truth parameters though)
    obs = utils.obs_data(true_params, seed=observation_seed, duration=duration)
    obs_stats = utils.obs_stats(true_params, seed=observation_seed)

    # MCMC samples for comparison
    sam = np.load('results/glm/seed' + str(seed) + '/sam_' + str(seed) + '.npz')['arr_0']


    ##########
    # SNPE-A #
    ##########

    g = init_g(seed=seed)

    res = infer.CDELFI(g, 
                     obs=obs_stats, 
                     n_hiddens=n_hiddens, 
                     seed=seed, 
                     reg_lambda=reg_lambda,
                     pilot_samples=pilot_samples,
                     svi=svi,
                     n_components=n_components,
                     verbose=verbose,
                     prior_norm=prior_norm)

    t = timeit.time.time()

    logs_A, tds_A, posteriors_A = res.run(n_train=n_train, 
                        n_rounds=5, 
                        minibatch=minibatch, 
                        epochs=epochs)

    print(timeit.time.time() -  t)

    #samples_true = np.load(os.path.join('results/' + model_id, 'seed42', 'samples.npy'))
    all_mmds_A = calc_all_mmds(sam[:,3000:13000:2].T, 
                             n_samples=N, 
                             posteriors=posteriors_A, 
                             init_g=init_g,
                             rej=True)

    print('all_mmds', all_mmds_A)



    ##########
    # SNPE-B #
    ##########


    g = init_g(seed=seed)

    res_B = infer.SNPE(g, 
                     obs=obs_stats, 
                     n_hiddens=n_hiddens, 
                     seed=seed, 
                     reg_lambda=reg_lambda,
                     pilot_samples=pilot_samples,
                     svi=svi,
                     n_components=n_components,
                     verbose=verbose,
                     prior_norm=prior_norm)

    t = timeit.time.time()

    logs_B, tds_B, posteriors_B = res_B.run(n_train=n_train, 
                        n_rounds=5, 
                        minibatch=minibatch, 
                        epochs=epochs)

    print(timeit.time.time() -  t)

    #samples_true = np.load(os.path.join('results/' + model_id, 'seed42', 'samples.npy'))
    all_mmds_B = calc_all_mmds(sam[:,3000:13000:2].T, 
                             n_samples=N, 
                             posteriors=posteriors_B, 
                             init_g=init_g,
                             rej=True)

    print('all_mmds', all_mmds_B)



    #######
    # APT #
    #######



    if train_on_all:
        epochs = [epochs//(r+1) for r in range(n_rounds)]

    # control MAF seed
    rng = np.random
    rng.seed(seed)

    # generator
    g = init_g(seed=seed)

    # inference object
    res_C = infer.SNPEC(g,
                     obs=obs_stats,
                     n_hiddens=n_hiddens,
                     seed=seed,
                     reg_lambda=reg_lambda,
                     pilot_samples=pilot_samples,
                     svi=svi,
                     n_mades=n_mades, # providing this argument triggers usage of MAFs (vs. MDNs)
                     act_fun=act_fun,
                     mode=mode,
                     rng=rng,
                     batch_norm=batch_norm,
                     verbose=verbose,
                     prior_norm=prior_norm)

    # train
    t = timeit.time.time()

    logs_C, tds_C, posteriors_C = res_C.run(
                        n_train=n_train,
                        proposal='discrete',
                        moo='resample',
                        n_null = minibatch-1,
                        n_rounds=n_rounds,
                        train_on_all=train_on_all,
                        minibatch=minibatch,
                        epochs=epochs,
                        val_frac=val_frac,
                        silent_fail=True,
                        stop_on_nan=True)

    print(timeit.time.time() - t)

    #samples_true = np.load(os.path.join('results/' + model_id, 'seed42', 'samples.npy'))
    all_mmds_C = calc_all_mmds(sam[:,3000:13000:2].T, 
                             n_samples=N, 
                             posteriors=posteriors_C, 
                             init_g=init_g,
                             rej=True)

    print('all_mmds', all_mmds_C)


    #######
    # SNL #
    #######


    # control MAF seed
    rng = np.random
    rng.seed(seed)

    # explicit call to MAF constructor
    theta, x = g.gen(1)
    n_inputs, n_outputs  = x.size, theta.size
    model = ConditionalMaskedAutoregressiveFlow(
                    n_inputs=n_inputs,
                    n_outputs=n_outputs,
                    n_hiddens=n_hiddens,
                    act_fun=act_fun,
                    n_mades=n_mades,
                    mode=mode,
                    rng=rng)


    # generator
    g = init_g(seed=seed)

    # inference object
    inf = nde.SequentialNeuralLikelihood(SNLprior(g.prior),               # method to draw parameters  
                                         SNLmodel(g.model, g.summary).gen # method to draw summary stats
                                        )

    # train
    t = timeit.time.time()

    rng = np.random # control  
    rng.seed(seed)  # MCMC seed
    model = inf.learn_likelihood(obs_stats.flatten(), 
                                 model, 
                                 n_samples=n_train, 
                                 n_rounds=n_rounds, 
                                 train_on_all=True, 
                                 thin=10, 
                                 save_models=True, 
                                 logger=sys.stdout, 
                                 rng=rng)

    print(timeit.time.time() - t)

    network = inf.all_models[-1]
    prior = SNLprior(init_g(seed=seed).prior)
    init_pars = true_params

    log_post = lambda t: network.eval([t, obs_stats.flatten()]) + prior.eval(t)
    sampler = mcmc.SliceSampler(init_pars, log_post, thin=10)
    sampler.gen(100, rng=rng)  # burn in
    samples = sampler.gen(N, rng=rng)


    #samples_true = np.load(os.path.join('results/' + model_id, 'seed42', 'samples.npy'))
    all_mmds_L = calc_all_mmds_snl(sam[:,3000:13000:2].T, 
                             samples_snl=inf.all_ps[1:] + [samples])

    print('all_mmds', all_mmds_L)



    # save and plot

    np.save('results/glm/seed' + str(seed) + '/all_mmds_v2', 
             {'all_mmds_A' : all_mmds_A, 
              'all_mmds_B' : all_mmds_B, 
              'all_mmds_C' : all_mmds_C, 
              'all_mmds_L' : all_mmds_L          
             })

    plt.figure(figsize=(9,6))
    plt.plot(np.sqrt(all_mmds_A), '>:', color='c', label='SNPE-A')
    plt.plot(np.sqrt(all_mmds_B), 'p:', color='g', label='SNPE-B')
    plt.plot(np.sqrt(all_mmds_L), 'o:', color='r', label='SNL')
    plt.plot(np.sqrt(all_mmds_C), 'd-', color='k', label='APT')

    plt.xlabel('N')
    plt.ylabel('maximum mean discrepancy')
    plt.xticks([0,2,4], [5000, 10000, 25000])
    plt.title('GLM (10-dim, 5 rounds with N=5k each)')
    plt.legend()
    plt.show()


In [ ]:
lower=init_g(seed).prior.m-1*np.diag(init_g(seed).prior.S)
upper=init_g(seed).prior.m+1*np.diag(init_g(seed).prior.S)

fig, _ = plot_pdf(pdf1=posteriors_A[-1],
         pdf2=posteriors_B[-1],
         samples=sam,
         lims= [
          [lower[i], upper[i]] for i in range(10)            
         ],
         figsize=(16,16))
fig.suptitle('SNPE-A & SNPE-B vs. MCMC')

fig, _ = plot_pdf(pdf1=posteriors_A[-1],
         samples=posteriors_C[-1].gen(10000).T,
         lims= [
          [lower[i], upper[i]] for i in range(10)            
         ],
         figsize=(16,16))
fig.suptitle('SNPE-A vs. APT')

fig, _ = plot_pdf(pdf1=posteriors_A[-1],
         samples=inf.all_ps[-1].T,
         lims= [
          [lower[i], upper[i]] for i in range(10)            
         ],                  
         figsize=(16,16))
fig.suptitle('SNPE-A vs. SNL')

In [ ]:
"""
mmd_snpea = np.array([0.16374293, 0.01935329, 0.0042091,  0.00060067, 0.0029527])
mmd_snpeb = np.array([0.16689721, 0.09796158, 0.09905345, 0.10139064, 0.09843219])
mmd_snl = np.array([0.0519698,  0.06865336, 0.06759704, 0.04965283, 0.04916893])
mmd_snpec = np.array([0.13863246, 0.01514593, 0.00414106, 0.00475852, 0.00567665])


plt.figure(figsize=(9,6))
plt.plot(np.sqrt(mmd_snpea), '>:', color='c', label='SNPE-A')
plt.plot(np.sqrt(mmd_snpeb), 'p:', color='g', label='SNPE-B')
plt.plot(np.sqrt(mmd_snl), 'o:', color='r', label='SNL')
plt.plot(np.sqrt(mmd_snpec), 'd-', color='k', label='APT')

plt.xlabel('N')
plt.ylabel('maximum mean discrepancy')
plt.xticks([0,2,4], [5000, 10000, 25000])
plt.title('GLM (10-dim, 5 rounds with N=5k each)')
plt.legend()
plt.show()
"""
